<i>Copyright (c) Recommenders contributors.</i>

<i>Licensed under the MIT License.</i>

# Neural Collaborative Filtering

Neural Collaborative Filtering (NCF) is a well known recommendation algorithm that generalizes the matrix factorization problem with multi-layer perceptron. 

This notebook provides an example of how to utilize and evaluate NCF implementation in the `recommenders`. We use a smaller dataset in this example to run NCF efficiently with GPU acceleration.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import pandas as pd
import tensorflow as tf
# tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.models.ncf.dataset import Dataset as NCFDataset
from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import (
    map, ndcg_at_k, precision_at_k, recall_at_k
)
from recommenders.utils.notebook_utils import store_metadata

from datasets import outfits

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))


System version: 3.10.13 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:24:38) [MSC v.1916 64 bit (AMD64)]
Pandas version: 2.3.3
Tensorflow version: 2.20.0


c:\Users\zivil\anaconda3\envs\outfit\lib\site-packages\pandera\_pandas_deprecated.py:160: FutureWarning: Importing pandas-specific classes and functions from the
top-level pandera module will be **removed in a future version of pandera**.
If you're using pandera to validate pandas objects, we highly recommend updating
your import:

```
# old import
import pandera as pa

# new import
import pandera.pandas as pa
```

If you're using pandera to validate objects from other compatible libraries
like pyspark or polars, see the supported libraries section of the documentation
for more information on how to import pandera:

https://pandera.readthedocs.io/en/stable/supported_libraries.html

To disable this warning, set the environment variable:

```
export DISABLE_PANDERA_IMPORT_WARNING=True
```

  warnings.warn(_future_warning, FutureWarning)


Set the default parameters.

In [3]:
# top k items to recommend
TOP_K = 10

# Change data size as appropriate
OUTFITS_DATA_SIZE = '200'

# Model parameters
EPOCHS = 100
BATCH_SIZE = 256

SEED = 42

### 1. Download the dataset

In [46]:
# df = outfits.load_pandas_df(
#     header=["UserId", "Weather", "Clothing", "Event", "Season"],
#     filepath=f"datasets/csv/own_feature2.csv"
# )
# df['ClothingId'] = df['Clothing'].astype('category').cat.codes
# print(df)

df = pd.read_csv("datasets/csv/own_feature2.csv")
df['ClothingId'] = df['Clothing'].astype('category').cat.codes

print(df)

     UserId Weather Clothing          Event  Season  Rating  ClothingId
0         1   Sunny  T-shirt  Casual Outing  Summer     5.0          11
1         1     Hot   Shorts       Exercise  Summer     4.0           8
2         1   Rainy   Hoodie         School    Fall     3.0           2
3         1    Cold   Jacket         Travel  Winter     4.0           3
4         1   Sunny  Sweater   Family Event  Spring     5.0          10
..      ...     ...      ...            ...     ...     ...         ...
198      24     Hot   Hoodie         School  Summer     5.0           2
199      24   Sunny  T-shirt  Casual Outing  Spring     5.0          11
200      24    Cold    Pants           Work  Winter     4.0           6
201      24   Windy   Hoodie         School  Winter     4.0           2
202      24   Sunny   Shorts          Beach  Summer     5.0           8

[203 rows x 7 columns]


### 2. Split the data using the Spark splitter provided in utilities

In [47]:
train, test = python_random_split(
    df, 
    ratio=0.75
)
train = train[train['Rating'] > 0]
assert len(train) > 0, "STOP: Training set is empty immediately after splitting."

Filter out any users or items in the test set that do not appear in the training set.

In [48]:
test = test[test["UserId"].isin(train["UserId"].unique())]
test = test[test["ClothingId"].isin(train["ClothingId"].unique())]

train_sorted = train.sort_values(by="UserId")
test_sorted = test.sort_values(by="UserId")

Write datasets to csv files.

In [49]:
train_file = "./train.csv"
test_file = "./test.csv"
train_sorted.to_csv(train_file, index=False)
test_sorted.to_csv(test_file, index=False)

Generate an NCF dataset object from the data subsets.

In [50]:
data = NCFDataset(
  train_file=train_file, 
  test_file=test_file, 
  seed=SEED, 
  col_user="UserId", 
  col_item="ClothingId", 
  col_rating="Rating")

assert data.n_users > 0 and data.n_items > 0, "STOP: The NCFDataset object loaded no users or items."

INFO:recommenders.models.ncf.dataset:Indexing ./train.csv ...
INFO:recommenders.models.ncf.dataset:Indexing ./test.csv ...
INFO:recommenders.models.ncf.dataset:Indexing ./test_full.csv ...


### 3. Train the NCF model on the training data, and get the top-k recommendations for our testing data

NCF accepts implicit feedback and generates prospensity of items to be recommended to users in the scale of 0 to 1. A recommended item list can then be generated based on the scores. Note that this quickstart notebook is using a smaller number of epochs to reduce time for training. As a consequence, the model performance will be slighlty deteriorated. 

In [51]:
model = NCF (
    n_users=data.n_users, 
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=4,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=1,
    seed=SEED
)

c:\Users\zivil\anaconda3\envs\outfit\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '



In [52]:
with Timer() as train_time:
    model.fit(data)

print(model)
print("Took {} seconds for training.".format(train_time))

INFO:recommenders.models.ncf.ncf_singlenode:Epoch 1 [0.26s]: train_loss = 0.693041 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 2 [0.02s]: train_loss = 0.692855 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 3 [0.02s]: train_loss = 0.692803 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 4 [0.02s]: train_loss = 0.692630 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 5 [0.02s]: train_loss = 0.692366 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 6 [0.02s]: train_loss = 0.692062 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 7 [0.02s]: train_loss = 0.691922 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 8 [0.02s]: train_loss = 0.691633 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 9 [0.03s]: train_loss = 0.691147 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 10 [0.03s]: train_loss = 0.690847 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 11 [0.02s]: train_loss = 0.690568 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 12 [0.02s]: train_loss =

Took 2.4315 seconds for training.


In [11]:
with Timer() as test_time:
    users, items, preds = [], [], []
    item = list(train.ClothingId.unique())
    for user in train.UserId.unique():
        user = [user] * len(item) 
        users.extend(user)
        items.extend(item)
        preds.extend(list(model.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"UserId": users, "ClothingId":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["UserId", "ClothingId"], how="outer")
    all_predictions = merged[merged.Rating.isnull()].drop('Rating', axis=1)

print("Took {} seconds for prediction.".format(test_time))


print(all_predictions)

Took 0.0464 seconds for prediction.
     UserId Weather Clothing  ClothingId  prediction
1         1     NaN      NaN           1    0.491099
4         1     NaN      NaN           3    0.615275
5         1     NaN      NaN           4    0.558853
8         1     NaN      NaN           7    0.550855
11        1     NaN      NaN           9    0.447569
..      ...     ...      ...         ...         ...
271      20     NaN      NaN           5    0.369384
272      20     NaN      NaN           6    0.430236
274      20     NaN      NaN           8    0.523288
275      20     NaN      NaN           9    0.374776
276      20     NaN      NaN          10    0.572345

[128 rows x 5 columns]


### 4. Evaluate how well NCF performs

The ranking metrics are used for evaluation.

In [12]:
eval_map = map(test, all_predictions, col_prediction='prediction', k=TOP_K, col_user='UserId', col_item='ClothingId')
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K, col_user='UserId', col_item='ClothingId', col_rating='Rating')
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K, col_user='UserId', col_item='ClothingId')
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K, col_user='UserId', col_item='ClothingId')

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.494918
NDCG:	0.621373
Precision@K:	0.172222
Recall@K:	0.618519


In [13]:
# # Record results for tests - ignore this cell
# store_metadata("map", eval_map)
# store_metadata("ndcg", eval_ndcg)
# store_metadata("precision", eval_precision)
# store_metadata("recall", eval_recall)
# store_metadata("train_time", train_time.interval)
# store_metadata("test_time", test_time.interval)